<a href="https://colab.research.google.com/github/rahiakela/data-science-research-and-practice/blob/main/data-science-bookcamp/case-study-3--disease-outbreaks/01_clustering_data_into_groups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering data into groups

Clustering is the process of organizing data points into conceptually meaningful groups.

What makes a given group `conceptually meaningful`? 

There is no easy answer to that question. The usefulness of any clustered output is dependent on the task we’ve been assigned.